<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong></h3>
<strong>Detach EC2 Instance from Auto Scaling Group</strong></div>
</center><center>
<h2 id="Detach-EC2-Instance-from-Auto-Scaling-Group"><strong>Detach EC2 Instance from Auto Scaling Group</strong></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)&nbsp;Get AWS EC2 Instances All&nbsp;</p>
<p>2) Get AWS Auto Scaling Group Instances</p>
<p>3) AWS Detach Instances From AutoScaling Group</p>

<h3 id="Filter-All-AWS-EC2-Instances">Filter All AWS EC2 Instances</h3>
<p>Using unSkript's <strong>Filter All AWS EC2 Instances</strong> action we list all the EC2 instances for a given region.</p>
<ul>
<li><strong>Input parameters:</strong>&nbsp; <code>region</code></li>
<li><strong>Output variable:</strong>&nbsp; <code>instance_ids</code></li>
</ul>

In [5]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_filter_ec2_instances_printer(output):
    if output is None:
        return
    pprint.pprint({"Instances": output})


@beartype
def aws_filter_ec2_instances(handle, region: str) -> List:
    """aws_filter_ec2_instances Returns an array of instances.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region to filter instances.

        :rtype: Array of instances.
    """
    ec2Client = handle.client('ec2', region_name=region)
    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
    result = []
    for reservation in res:
        for instance in reservation['Instances']:
            result.append(instance['InstanceId'])
    return result


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not instance_id",
    "condition_result": true
    }''')
task.configure(outputName="instance_id")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_ec2_instances, lego_printer=aws_filter_ec2_instances_printer, hdl=hdl, args=args)

<h3 id="List-Publicly-Accessible-RDS-Instances">Get AWS Auto Scaling Group Instances</h3>
<p>Using unSkript's&nbsp;<strong>Get AWS Auto Scaling Group Instances</strong> action, we find out all the AWS EC2 instances attached to Auto Scaling Group by passing instance ids from Step 1.</p>
<ul>
<li><strong>Input parameters:</strong>&nbsp; <code>region, instance_ids</code></li>
<li><strong>Output variable:</strong>&nbsp; <code> auto_scale_instances</code></li>
</ul>

In [26]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from tabulate import tabulate
from botocore.exceptions import ClientError
import pprint

from beartype import beartype
@beartype
def aws_get_auto_scaling_instances_printer(output):
    if output is None:
        return
    print(tabulate(output, headers='keys'))


@beartype
def aws_get_auto_scaling_instances(handle, instance_ids: list, region: str) -> List:
    """aws_get_auto_scaling_instances List of Dict with instanceId and attached groups.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type instance_ids: list
        :param instance_ids: List of instances.

        :type region: string
        :param region: Region to filter instances.

        :rtype: List of Dict with instanceId and attached groups.
    """
    result = []
    ec2Client = handle.client('autoscaling', region_name=region)
    try:
        response = ec2Client.describe_auto_scaling_instances(InstanceIds=instance_ids)
        for group in response["AutoScalingInstances"]:
            group_dict = {}
            group_dict["InstanceId"] = group["InstanceId"]
            group_dict["AutoScalingGroupName"] = group["AutoScalingGroupName"]
            result.append(group_dict)
    except Exception as error:
        err = {"Error":error}
        result.append(err)
    return result


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "instance_ids": "instance_id",
    "region": "region"
    }''')
task.configure(outputName="auto_scale_instances")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_auto_scaling_instances, lego_printer=aws_get_auto_scaling_instances_printer, hdl=hdl, args=args)

<h3 id="Modify-Snapshot-Output">Check Auto Scaling Group</h3>
<p>In this action, we check for the auto-scaling group name in the step 2 output and compare the group name with the given group name from the parameter.</p>
<blockquote>
<p>Output variable: asg_instance</p>
</blockquote>

In [29]:
asg_instance  = []
if asg_name:
    for instance in auto_scale_instances:
        if asg_name in instance["AutoScalingGroupName"]:
            asg_instance.append(instance)

<h3 id="List-Publicly-Accessible-RDS-Instances">Detach AWS Instances From ASG</h3>
<p>Using unSkript's&nbsp;<strong>Detach AWS Instance From ASG</strong> action. we detach the AWS instance from Auto Scaling Group by passing the data from step 2.</p>
<ul>
<li><strong>Input parameters:</strong>&nbsp; <code>region, group_name, instance_ids</code></li>
<li><strong>Output variable:</strong> &nbsp;<code>detach_asg_instances</code></li>
</ul>

In [4]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint

from beartype import beartype
@beartype
def aws_detach_autoscaling_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_detach_autoscaling_instances(
    handle,
    instance_ids: str,
    group_name: str,
    region: str
) -> Dict:
    """aws_detach_autoscaling_instances detach instances from autoscaling group.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type instance_ids: string
        :param instance_ids: Name of instances.

        :type group_name: string
        :param group_name: Name of AutoScaling Group.

        :type region: string
        :param region: AWS Region of autoscaling group.

        :rtype: Dict with the detach instance info.
    """

    ec2Client = handle.client("autoscaling", region_name=region)
    result = {}
    try:
        if instance_ids in Instance_IDs:
            response = ec2Client.detach_instances(
                InstanceIds=[instance_ids],
                AutoScalingGroupName=group_name,
                ShouldDecrementDesiredCapacity=True
                )
            result = response
        else:
            result["error"] = "Instance not match"
    except Exception as error:
        result["error"] = error

    return result


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "group_name": "iter.get(\\"AutoScalingGroupName\\")",
    "instance_ids": "iter.get(\\"InstanceId\\")",
    "region": "region"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "asg_instance",
    "iter_parameter": ["group_name","instance_ids"]
    }''')
task.configure(outputName="detach_asg_instances")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_detach_autoscaling_instances, lego_printer=aws_detach_autoscaling_instances_printer, hdl=hdl, args=args)

### Conclusion
In this Runbook, we demonstrated the use of unSkript's AWS legos to Detach AWS instance from ASG. This runbook help to detach the given instance from Auto Scalling Group. To view the full platform capabilities of unSkript please visit https://unskript.com